In [2]:
import pandas as pd
from pathlib import Path

# Load the raw data
path = Path("data/sunlight_region.csv")
df = pd.read_csv(path)

In [3]:
df.head()

,id,year,region,nervous,calm,down,happy,depressed,income,msa,urban_rural,prev_region,moved,mh_index,sunlight,sunlight_lag,sunlight_change,sunlight_increase,sunlight_move_interaction
0,1.0,2000,1.0,3.0,3.0,3.0,3.0,3.0,128000.0,3.0,1.0,NaN,0,3.0,3.5,NaN,NaN,0,NaN
1,1.0,2001,1.0,NaN,NaN,NaN,NaN,NaN,177000.0,3.0,1.0,1.0,0,NaN,3.5,3.5,0.0,0,0.0
2,1.0,2002,1.0,1.0,4.0,2.0,3.0,2.0,128000.0,3.0,1.0,1.0,0,2.4,3.5,3.5,0.0,0,0.0
3,1.0,2003,1.0,NaN,NaN,NaN,NaN,NaN,185000.0,3.0,1.0,1.0,0,NaN,3.5,3.5,0.0,0,0.0
4,1.0,2004,1.0,1.0,4.0,1.0,4.0,1.0,200000.0,3.0,1.0,1.0,0,2.2,3.5,3.5,0.0,0,0.0


In [7]:
df.drop(columns='mh_index', inplace=True)

In [8]:
df.head(0)

,id,year,region,nervous,calm,down,happy,depressed,income,msa,urban_rural,prev_region,moved,sunlight,sunlight_lag,sunlight_change,sunlight_increase,sunlight_move_interaction


In [10]:
import numpy as np# Columns
neg_cols = ["nervous", "down", "depressed"]   # negative → higher value should mean better MH
pos_cols = ["happy", "calm"]                  # positive → already oriented correctly

# Reverse-score negative items: 1 → 4, 2 → 3, 3 → 2, 4 → 1
for col in neg_cols:
    df[col + "_rev"] = df[col].apply(lambda x: 5 - x if pd.notnull(x) else np.nan)

# Create the MH index (mean of all items)
df["mh_index"] = df[[c + "_rev" for c in neg_cols] + pos_cols].mean(axis=1)

In [11]:
df.head()

,id,year,region,nervous,calm,down,happy,depressed,income,msa,...,moved,sunlight,sunlight_lag,sunlight_change,sunlight_increase,sunlight_move_interaction,nervous_rev,down_rev,depressed_rev,mh_index
0,1.0,2000,1.0,3.0,3.0,3.0,3.0,3.0,128000.0,3.0,...,0,3.5,NaN,NaN,0,NaN,2.0,2.0,2.0,2.4
1,1.0,2001,1.0,NaN,NaN,NaN,NaN,NaN,177000.0,3.0,...,0,3.5,3.5,0.0,0,0.0,NaN,NaN,NaN,NaN
2,1.0,2002,1.0,1.0,4.0,2.0,3.0,2.0,128000.0,3.0,...,0,3.5,3.5,0.0,0,0.0,4.0,3.0,3.0,3.4
3,1.0,2003,1.0,NaN,NaN,NaN,NaN,NaN,185000.0,3.0,...,0,3.5,3.5,0.0,0,0.0,NaN,NaN,NaN,NaN
4,1.0,2004,1.0,1.0,4.0,1.0,4.0,1.0,200000.0,3.0,...,0,3.5,3.5,0.0,0,0.0,4.0,4.0,4.0,4.0


In [12]:
df.shape

(98824, 22)

In [13]:
df.to_csv("nlsy_cleaned.csv", index=False)